# Final Project: Challenge 5
Contributions From: Ryan Cottone

**Difficulty**: <span style="color:orange">$\bullet \bullet$</span>

Welcome to the 5th challenge!

We've managed to leak the server's old RSA private key. However, they detected this and switched to a new one.

Similarly to Challenge 4, you will be taking on the role of an active attacker by implementing an Interceptor class.

Recover the user's password.

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
%run client.ipynb
%run server.ipynb
from utils import *

In [ ]:
ellipticCurve = [0, 7]
ellipticCurveModulus = 115792089237316195423570985008687907853269984665640564039457584007908834671663 
generatorPoint = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)
generatorPointOrder =115792089237316195423570985008687907852837564279074904382605163141518161494337

A copy of the server instance which we were able to hack, but is now outdated..

In [ ]:
OLD_SERVER_RSA_PUBLIC_KEY = (65537, 569587199484841303316658210815764428763985421987409075188989984797014449064633668839243553137738860233963515168449435430268386255471489721419848705738073)
OLD_SERVER_RSA_PRIVATE_KEY = 146262156951499066077422540394258198752904628936724404936989982058210723153453931962858005026722533458178756916842060121732362272046754119095094165549589
OLD_SERVER_CERTIFICATE = 2609386874937173177930731230739036449322590649208534257580386986889318952529669701135763767678586130303402952908012530234093564487126812457745919925043366

In [ ]:
class HackedServer:
    RSA_PUBLIC_KEY = OLD_SERVER_RSA_PUBLIC_KEY
    
    RSA_PRIVATE_KEY = OLD_SERVER_RSA_PRIVATE_KEY
    
    SIGNED_SERVER_CERTIFICATE = OLD_SERVER_CERTIFICATE
    
    ELLIPTIC_CURVE = [0,7]
    
    ELLIPTIC_CURVE_MODULUS = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
    
    GENERATOR_POINT = (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
    
    GENERATOR_POINT_ORDER = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
    
    LOGINS = {}
    
    def signMessage(self, message):
        return signRSA(self.RSA_PRIVATE_KEY, message, self.RSA_PUBLIC_KEY[1])
    
    def publishRSAPublicKey(self):
        """
        Returns ((e, N), signature) where e,N are the server's public key, and signature is the cert authorities' signature 
        on e||N.
        """
        
        return ((self.RSA_PUBLIC_KEY[0], self.RSA_PUBLIC_KEY[1]), self.SIGNED_SERVER_CERTIFICATE)
    
    def generateECDHParameters(self):
        """
        Returns a generator point P, curve E, and modulus N signed via the server's RSA keys.
        """

        intToSign = int(str(self.ELLIPTIC_CURVE[0]) + str(self.ELLIPTIC_CURVE[1]) 
                        + str(self.ELLIPTIC_CURVE_MODULUS) + str(self.GENERATOR_POINT[0]) + str(self.GENERATOR_POINT[1]))

        signature = signRSA(self.RSA_PRIVATE_KEY, intToSign, self.RSA_PUBLIC_KEY[1]) 

        return self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT, self.GENERATOR_POINT_ORDER, signature
    
    def generateSignedECDHMessage(self):
        nP, n = generateECDH(self.GENERATOR_POINT, self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT_ORDER)
        
        intToSign = int(str(nP[0]) + str(nP[1]))
        
        self.ECDH_SECRET = n
        
        return nP, self.signMessage(intToSign)
    
    def verifyECDHMessage(self, P, signature, publicKey):
        """
        Verifies point P was signed by the given publicKey (e, N).
        """

        expected = int(str(P[0]) + str(P[1]))

        return verifyRSA(publicKey[0], signature, publicKey[1], expected)
    
    def acceptConnection(self, clientPublicKey):
        self.CLIENT_PUBLIC_KEY = clientPublicKey
    
    def acceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        
        assert self.verifyECDHMessage(P, signature, self.CLIENT_PUBLIC_KEY), "Invalid signature on received ECDH message."
        
        # Signature verified
        sharedSecretPoint = doubleAndAdd(P, self.ECDH_SECRET, self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS)
        
        self.SHARED_SECRET_INT = pointToMessage(sharedSecretPoint)
        
        return True
    
    def createLogin(self, username, password):
        if username in self.LOGINS: 
            return False
        
        salt = ''.join(np.random.choice(list(string.ascii_lowercase), 5))
        
        intToHash = textToInt(password + salt)
        
        hashedPassword = hash_SHA256(intToHash)
        
        
        self.LOGINS[username] = (hashedPassword, salt)
        
        return True
        
    def verifyLogin(self, username, password):
        assert username in self.LOGINS, "Username does not exist"
        
        hashedPassword = self.LOGINS[username][0]
        salt = self.LOGINS[username][1] 
        
        intToHash = textToInt(password + salt)
                
        return hashedPassword == hash_SHA256(intToHash)

    def encryptMessage(self, message):
        """
        Given string message, encrypts with the current setup
        """
        
        return encryptAES(self.SHARED_SECRET_INT, bytes(message))
    
    def decryptMessage(self, encrypted):
        """
        Given encrypted message, decrypts with the current setup
        """
        
        return decryptAES(self.SHARED_SECRET_INT, encrypted)
    
    def verifyRequestIntegrity(self, requestedData):
        """
        Takes in a request of the form { data: ..., hmac: ...} and returns True if the HMAC is valid.
        """
        
        return verifyHMAC(self.SHARED_SECRET_INT, bytes(requestedData['data'], 'utf-8'), requestedData['hmac'])
    
    def handleRequest(self, requestData):
        if 'data' not in requestData or 'hmac' not in requestData:
            raise AssertionError("Invalid requested format")
            
        assert self.verifyRequestIntegrity(requestData), "Request MAC check failed"
        
        # Request verified, time to decrypt
        request = json.loads(self.decryptMessage(requestData['data']))
        
        if request['type'] == 'createLogin':
            assert 'username' in request and 'password' in request, "Bad request data"
            
            return self.createLogin(request['username'], request['password'])
        elif request['type'] == 'verifyLogin':
            return self.verifyLogin(request['username'], request['password'])
        else:
            raise AssertionError("Invalid request type")

### Fill this in:

In [ ]:
class Interceptor:
    def __init__(self):
        self.clientPassword = None
        self.hackedServer = HackedServer()
        
    def interceptMessage(self, message):
        """
        The message will be of the form 
        { 
            sender: 'client/server',
            receiver: 'client/server',
            type: 'certificate/ECDH_Parameters/ECDH_Message/request',
            data: ...
        }
        """
        messageType = message['type']
        
        if messageType == 'CLIENT_RSA':
            return self.hackedServer.acceptConnection(message['data'])
        elif messageType == 'certificate':
            return self.hackedServer.publishRSAPublicKey()
        elif messageType == 'ECDH_Parameters':
            return ...
        elif messageType == 'ECDH_Message':
            if message['sender'] == 'client':
                ...
            else:
                return ...
        elif messageType == 'request':
            if message['sender'] != 'client':
                pass
            firstRequestData = message['data']['data'] # Request data
            
            decryptedObj = ...
            
            decrypted = json.loads(decryptedObj)
    
            self.CLIENT_PASSWORD = decrypted['password']
        
            return None
        else:
            print('err', message)
            return ()
        
    def getPassword(self):
        return self.CLIENT_PASSWORD

Here's an example to test with:

In [ ]:
random.seed(0)

# New client
client = Client()

# Client initiates server connection
server = Server()

# Initiate Interceptor
intercept = Interceptor()

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'CLIENT_RSA', 'data': client.RSA_PUBLIC_KEY})
if message:
    server.acceptConnection(*message)


server_PK, signature = server.publishRSAPublicKey()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'certificate', 'data': (server_PK, signature)})
assert client.verifyServerCertificate(*message)

# Once verified, ask for ECDH parameters
ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Parameters', 'data': (ellipticCurve, ellipticCurveModulus,
                                                                                generatorPoint, generatorPointOrder, signature)})

assert client.verifyECDHParameters(*message)

# Once ECDH parameters are verified, generate our secret + public value
server_ECDH = server.generateSignedECDHMessage()

messageOne = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Message', 'data': server_ECDH})
client_ECDH = client.generateSignedECDHMessage()

messageTwo = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'ECDH_Message', 'data': client_ECDH})
if messageOne:
    client.acceptECDHMessage(*messageOne)
if messageTwo:
    server.acceptECDHMessage(*messageTwo)


signupRequest = {'data': '{"iv": "EPElhA5Q01yLaN/ll816IA==", "ciphertext": "tsZnuIwRyoFuqGphxL5KtLCdmAAyVlH1xdZrfXlIy3MIbf/Fo09zzm/Fcg+JINlUb310ezB17qirC+Qjrv0vDeAWFxMWpE0Jw8Ut4zNMsfU="}',
 'hmac': 'a6d3633e63bcee48790db5ab0cc512c4e01e3ce809260d0d4a13399f30f5f40a'}

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'request', 'data': signupRequest})

# By now, we should have access to the client's password.
print('Recovered the following password:', intercept.getPassword())

### Submission

Once you've recovered the password, enter it into the Gradescope page [here](https://www.gradescope.com/courses/406561/assignments/2458792) and copy-paste your **Interceptor** class in the same assignment. (Unfortunately our usual autograder does not play nice with this style of challenge)